In [1]:
import pandas as pd
import numpy as np
import seaborn as sns # For Data Visualization
import matplotlib.pyplot as plt # For Data Visualization

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [23]:
data=pd.read_csv("/content/drive/MyDrive/finance_complaint/consumer_complaints.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
data=data

In [21]:
data.head(3)

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473


In [26]:
data.loc[(data['consumer_complaint_narrative'].isnull()==False) & (data['company_public_response'].isnull()==False)].head(3)

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
190237,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,Experian is reporting my OPEN and CURRENT Mort...,Company chooses not to provide a public response,Experian,CA,900XX,Older American,Consent provided,Web,03/25/2015,Closed with explanation,Yes,No,1290254
190263,03/19/2015,Credit card,NaN,Other,NaN,i opened XXXX Bank of America credit cards 15-...,Company chooses not to provide a public response,Bank of America,OR,971XX,NaN,Consent provided,Web,03/19/2015,Closed with monetary relief,Yes,Yes,1292209
191294,03/19/2015,Debt collection,Mortgage,False statements or representation,Impersonated an attorney or official,Lawyers at this law firm ( XXXX ) sent me a le...,Company believes it acted appropriately as aut...,"Dean Morris, L.L.P.",LA,708XX,NaN,Consent provided,Web,04/14/2015,Closed with explanation,Yes,Yes,1290181


In [31]:
df=data.drop(['issue','company_public_response','consumer_complaint_narrative','sub_issue','complaint_id','company_response_to_consumer','date_sent_to_company','zipcode','date_received'],axis=1)

In [32]:
df.head(3)

,product,sub_product,company,state,tags,consumer_consent_provided,submitted_via,timely_response,consumer_disputed?
0,Mortgage,Other mortgage,U.S. Bancorp,CA,NaN,NaN,Referral,Yes,Yes
1,Mortgage,Other mortgage,Wells Fargo & Company,CA,NaN,NaN,Referral,Yes,Yes
2,Credit reporting,NaN,Wells Fargo & Company,NY,NaN,NaN,Postal mail,Yes,No


In [34]:
df.shape

(555957, 9)

In [35]:
train_X=df.drop('consumer_disputed?',axis=1)
train_y=df['consumer_disputed?']

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555957 entries, 0 to 555956
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   product                    555957 non-null  object
 1   sub_product                397635 non-null  object
 2   company                    555957 non-null  object
 3   state                      551070 non-null  object
 4   tags                       77959 non-null   object
 5   consumer_consent_provided  123458 non-null  object
 6   submitted_via              555957 non-null  object
 7   timely_response            555957 non-null  object
 8   consumer_disputed?         555957 non-null  object
dtypes: object(9)
memory usage: 38.2+ MB


**Creating Pipeline**

In [45]:

from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

categorical_transformer=Pipeline(steps=[
            ('imputer',SimpleImputer(strategy='most_frequent')),
            ('onehot',OneHotEncoder(handle_unknown='ignore'))
])
#numerical_transformer=Pipeline(steps=[
          #    ('imputer', SimpleImputer(strategy='median'))                       
#])

preprocessor=ColumnTransformer(transformers=[
    ('cat',categorical_transformer,train_X.columns)
])


In [42]:
from sklearn.preprocessing import LabelEncoder

labels=df['consumer_disputed?']
labelencoder= LabelEncoder()
train_y=labelencoder.fit_transform(labels)

In [43]:
train_y

array([1, 1, 0, ..., 0, 0, 0])

In [46]:
from xgboost import XGBClassifier

xgboost_model_ = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')

fin_pipeline=Pipeline([
    ('preprocessor',preprocessor),
     ("model", xgboost_model_)
])

fin_pipeline.fit(train_X,train_y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['product', 'sub_product', 'company', 'state', 'tags',
       'consumer_consent_provided', 'submitted_via', 'timely_response'],
      dtype='object'))])),
                ('model',
                 XGBClassifier(learning_rate=1, max_depth=2, n_estimators=2))])